In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import math
from collections import defaultdict
import csv
import pandas as pd
import numpy as np
from flask import jsonify
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.externals import joblib
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
data = pd.read_csv('house_prediction/data/output copy.csv')

# Separate out the x_data and y_data.
data_X = data.loc[:, [col for col in data.columns if col not in ["price_sqft","address","url","price"]]]
data_y = data.loc[:, "price"]
#print data_X

In [3]:
""" about label encoder
#https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn
# Encoding the variable
fit = df.apply(lambda x: d[x.name].fit_transform(x))
# Inverse the encoded
fit.apply(lambda x: d[x.name].inverse_transform(x))
# Using the dictionary to label future data
df.apply(lambda x: d[x.name].transform(x))
"""
encode_list = ['city','state']
d = defaultdict(LabelEncoder)
data_X_encoded = data_X.apply(lambda x: d[x.name].fit_transform(x) if x.name in encode_list else x)
#print data_X_encoded

In [4]:
print d['city']
np.save('city.npy', d['city'].classes_)
np.save('state.npy', d['state'].classes_)
le_city = LabelEncoder()
le_city.classes_ = np.load('city.npy')
le_state = LabelEncoder()
le_state.classes_ = np.load('state.npy')
print le_city.transform(['Atlanta','Brookhaven','College Park','Dunwoody','Marietta','Sandy Springs','Smyrna'])


LabelEncoder()
[0 1 2 3 4 5 6]


In [5]:
#input format: latitude	longitude	city	state	postal_code
#  #bedrooms	#bathrooms	sqft	d1	d2	d3	d4	d5	d6	d7	d8	d9	d10	d11	d12	d13	d14	d15	d16	d17	d18	d19	d20
x_test = [33.913258,-84.451048,le_city.transform(['Atlanta'])[0],
          le_state.transform(['GA'])[0],30339,5,7,8407]


In [6]:
from sklearn.metrics import fbeta_score, make_scorer
def my_custom_loss_func(ground_truth, predictions):
    diff = -np.abs((ground_truth - predictions)/ground_truth).mean()
    return diff
my_scorer = make_scorer(my_custom_loss_func, greater_is_better=True)
cv_para = ShuffleSplit(n_splits=10, test_size=0.1, random_state=100)

In [ ]:
nn_model = MLPRegressor(hidden_layer_sizes=(1, ), activation='relu',
             solver='adam', alpha=0.00001, batch_size='auto',
             learning_rate='constant', learning_rate_init=0.001,
             power_t=0.5, max_iter=500, shuffle=True,
             random_state=None, tol=0.001, verbose=False, 
             warm_start=False, momentum=0.9, nesterovs_momentum=True, 
             early_stopping=False, validation_fraction=0.1,
             beta_1=0.9, beta_2=0.999, epsilon=1e-08)


#print 'cross validation score of rf_model: '+str(cross_val_score(nn_model, data_X_encoded, data_y,n_jobs=3,verbose=5,cv=cv_para, scoring=my_scorer).mean())
   


In [8]:
from sklearn.svm import SVC
svc_model = SVC()
print 'cross validation score of rf_model: '+str(cross_val_score(svc_model, data_X_encoded, data_y,n_jobs=3,verbose=5,cv=cv_para, scoring=my_scorer).mean())
   


[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] .......................... , score=-0.445100452412, total= 1.9min
[CV]  ................................................................
[CV] .......................... , score=-0.470782390186, total= 1.9min
[CV]  ................................................................
[CV] .......................... , score=-0.452445768503, total= 2.0min
[CV]  ................................................................
[CV] ............................ , score=-0.4398560679, total= 2.0min
[CV]  ................................................................
[CV] .......................... , score=-0.421575860156, total= 2.0min
[CV]  ................................................................
[CV] .......................... , score=-0.442935371971, total= 2.0min
[CV]  

[Parallel(n_jobs=3)]: Done   8 out of  10 | elapsed:  6.2min remaining:  1.5min


[CV] .......................... , score=-0.455498666279, total= 2.2min
[CV] .......................... , score=-0.432285492063, total= 1.3min
cross validation score of rf_model: -0.4449395680566049


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:  7.5min finished


In [7]:
from sklearn.preprocessing import StandardScaler, normalize
scaler = StandardScaler()
scaler.fit(data_X_encoded)
data_X_encoded_p = scaler.transform(data_X_encoded)
print data_X_encoded_p

[[ 0.17792227  0.54697493 -0.39687684 ... -1.14964386 -1.1644691
  -0.37638128]
 [ 1.33052822 -0.77654031 -0.39687684 ...  0.75357237  0.85642363
  -1.67750066]
 [-0.3522976   0.6008744  -0.39687684 ... -0.93166051 -1.13190351
   0.21403126]
 ...
 [ 0.96056049 -1.64351593  3.1885446  ...  1.2120073   1.05967078
  -1.57081208]
 [ 1.07629065 -1.28557789  3.1885446  ...  0.96539354  0.88456719
  -1.88587205]
 [-0.17704909 -0.33560701 -0.39687684 ... -0.24112643 -0.80777282
  -0.53490594]]


In [10]:
parameters = {'C':[0.001,0.1,10]}
clf = GridSearchCV(svc_model, parameters,scoring=my_scorer,cv=cv_para,verbose=30,n_jobs=3)
clf.fit(data_X_encoded_p,data_y)
print ("rf best para:",clf.best_params_)
print ("rf best score:",clf.best_score_)
print ("results",clf.cv_results_)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
[CV] C=0.001 .........................................................
[CV] C=0.001 .........................................................
[CV] C=0.001 .........................................................
[CV] ................... C=0.001, score=-0.548685773639, total= 1.6min
[CV] C=0.001 .........................................................


[Parallel(n_jobs=3)]: Done   1 tasks      | elapsed: 13.2min


[CV] ................... C=0.001, score=-0.552738766195, total= 1.7min
[CV] C=0.001 .........................................................


[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed: 13.4min


[CV] ................... C=0.001, score=-0.555131516748, total= 1.7min
[CV] C=0.001 .........................................................


...

[CV] ...................... C=10, score=-0.283317941585, total= 1.9min
[CV] C=10 ............................................................
[CV] ...................... C=10, score=-0.282718548203, total= 2.1min
[CV] C=10 ............................................................


[Parallel(n_jobs=3)]: Done  27 out of  30 | elapsed: 122.6min remaining: 13.6min


[CV] ...................... C=10, score=-0.300966422183, total= 1.8min
[CV] ...................... C=10, score=-0.279754663472, total= 1.8min
[CV] ...................... C=10, score=-0.298554558536, total= 2.0min


[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed: 136.4min finished


('rf best para:', {'C': 10})
('rf best score:', -0.29374962551542405)
('results', {'rank_test_score': array([3, 2, 1], dtype=int32), 'split6_test_score': array([-0.55097746, -0.5186742 , -0.28271855]), 'split7_train_score': array([-0.54306008, -0.50473224, -0.26990705]), 'split0_train_score': array([-0.54092719, -0.53135959, -0.26207476]), 'split2_test_score': array([-0.55273877, -0.54593748, -0.2973243 ]), 'mean_fit_time': array([23.0485775 , 27.40124047, 32.89755569]), 'split3_train_score': array([-0.54170212, -0.53037595, -0.26812942]), 'param_C': masked_array(data=[0.001, 0.1, 10],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'split6_train_score': array([-0.54138875, -0.51006339, -0.2698514 ]), 'split9_test_score': array([-0.53216879, -0.50079608, -0.29855456]), 'std_test_score': array([0.00831368, 0.01412797, 0.01231559]), 'params': [{'C': 0.001}, {'C': 0.1}, {'C': 10}], 'std_fit_time': array([0.66708845, 1.10692699, 2.36115997]), 'std

In [8]:
nn_model = MLPRegressor(hidden_layer_sizes=(1, ), activation='relu',
             solver='adam', alpha=0.00001, batch_size='auto',
             learning_rate='constant', learning_rate_init=0.001,
             power_t=0.5, max_iter=1000, shuffle=True,
             random_state=None, tol=0.0001, verbose=False, 
             warm_start=False, momentum=0.9, nesterovs_momentum=True, 
             early_stopping=False, validation_fraction=0.1,
             beta_1=0.9, beta_2=0.999, epsilon=1e-08)
parameters = {'hidden_layer_sizes':[(2,),(20,),(50,),(100,)],'alpha':[0.0001,0.01,1.0,100]}
clfridgenn = GridSearchCV(nn_model, parameters,scoring=my_scorer,cv=cv_para,verbose=30,n_jobs=3)
clfridgenn.fit(data_X_encoded_p,data_y)
print ("nn best para:",clfridgenn.best_params_)
print ("nn best score:",clfridgenn.best_score_)
print ("results",clfridgenn.cv_results_)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV] alpha=0.0001, hidden_layer_sizes=(2,) ...........................
[CV] alpha=0.0001, hidden_layer_sizes=(2,) ...........................
[CV] alpha=0.0001, hidden_layer_sizes=(2,) ...........................


/Users/wqshen/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  alpha=0.0001, hidden_layer_sizes=(2,), score=-0.999799980276, total=  15.3s
[CV] alpha=0.0001, hidden_layer_sizes=(2,) ...........................


/Users/wqshen/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  alpha=0.0001, hidden_layer_sizes=(2,), score=-0.999801029937, total=  15.5s


...

[Parallel(n_jobs=3)]: Done 155 tasks      | elapsed: 29.0min


[CV]  alpha=100, hidden_layer_sizes=(100,), score=-0.480049145354, total=  57.1s
[CV] alpha=100, hidden_layer_sizes=(100,) ............................
[CV]  alpha=100, hidden_layer_sizes=(100,), score=-0.468153754054, total= 1.0min
[CV] alpha=100, hidden_layer_sizes=(100,) ............................
[CV]  alpha=100, hidden_layer_sizes=(100,), score=-0.484410879618, total= 1.0min
[CV]  alpha=100, hidden_layer_sizes=(100,), score=-0.468494326792, total=  58.6s
[CV]  alpha=100, hidden_layer_sizes=(100,), score=-0.478642005746, total=  43.1s


[Parallel(n_jobs=3)]: Done 160 out of 160 | elapsed: 30.6min finished


('nn best para:', {'alpha': 0.01, 'hidden_layer_sizes': (100,)})
('nn best score:', -0.46740761595770186)
('results', {'std_train_score': array([0.04454563, 0.00957797, 0.00859853, 0.00347943, 0.02511375,
       0.02607913, 0.01002308, 0.00408969, 0.03436148, 0.01018604,
       0.01767343, 0.00439132, 0.04122141, 0.008993  , 0.00669918,
       0.00438139]), 'split4_test_score': array([-0.89101314, -0.61681973, -0.57809212, -0.50188084, -0.89124226,
       -0.61973556, -0.57507691, -0.5049513 , -0.92761329, -0.59615305,
       -0.5818548 , -0.50641061, -0.94411888, -0.60253835, -0.57340979,
       -0.50555101]), 'mean_train_score': array([-0.9327198 , -0.60577756, -0.54777888, -0.47401115, -0.93228452,
       -0.61517582, -0.55414484, -0.47176379, -0.95214974, -0.59036512,
       -0.5619876 , -0.47772272, -0.9269756 , -0.60364113, -0.55474923,
       -0.47907297]), 'param_hidden_layer_sizes': masked_array(data=[(2,), (20,), (50,), (100,), (2,), (20,), (50,), (100,),
                   (

/Users/wqshen/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [11]:
ridge1_model = Ridge(alpha=100, fit_intercept=True, normalize=False, copy_X=True, 
      max_iter=None, tol=0.001, solver='auto', random_state=None)
print 'cross validation score of rf_model: '+str(cross_val_score(ridge1_model, data_X_encoded_p, data_y,n_jobs=3,verbose=5,cv=cv_para, scoring=my_scorer).mean())
   


[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ........................... , score=-0.43721966753, total=   0.1s
[CV] .......................... , score=-0.385164567192, total=   0.1s
[CV] .......................... , score=-0.422900174776, total=   0.1s
[CV]  ................................................................
[CV]  ................................................................
[CV] .......................... , score=-0.392348908909, total=   0.0s
[CV]  ................................................................
[CV] .......................... , score=-0.444480902582, total=   0.0s
[CV]  ................................................................
[CV]  ................................................................
[CV] .......................... , score=-0.422893858518, total=   0.0s
[CV] .

[Parallel(n_jobs=3)]: Done   5 out of  10 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=3)]: Done   8 out of  10 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.3s finished


In [12]:
parameters = {'alpha':[0.0001,0.001,0.01,0.1,1,10,100]}
clfridge1 = GridSearchCV(ridge1_model, parameters,scoring=my_scorer,cv=cv_para,verbose=30,n_jobs=3)
clfridge1.fit(data_X_encoded_p,data_y)
print ("rf best para:",clfridge1.best_params_)
print ("rf best score:",clfridge1.best_score_)
print ("results",clfridge1.cv_results_)

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] .............. alpha=0.0001, score=-0.392134866489, total=   0.1s
[CV] .............. alpha=0.0001, score=-0.354291748588, total=   0.1s
[CV] .............. alpha=0.0001, score=-0.408031453265, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] .............. alpha=0.0001, score=-0.365581443549, total=   0.0s
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] .............. alpha=0.0001, score=-0.402000848892, total=   0.0s
[CV] .............. alpha=0.0001, score=-0.385072546889, total=   0.0s
[CV] alpha=0.000

[Parallel(n_jobs=3)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Batch computation too fast (0.1433s.) Setting batch_size=2.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Batch computation too fast (0.0703s.) Setting batch_size=10.
[Parallel(n_jobs=3)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done  14 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done  18 tasks      | elapsed:    0.4s


[CV] alpha=0.001 .....................................................
[CV] alpha=0.01 ......................................................
[CV] ................. alpha=0.001, score=-0.3757350559, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] alpha=0.1 .......................................................
[CV] ................ alpha=0.01, score=-0.377219467288, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ............... alpha=0.001, score=-0.393406580493, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ................ alpha=0.01, score=-0.394640326952, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ................ alpha=0.01, score=-0.392795241618, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ................. alpha=0.1, score=-0.381556185517, total=   0.0s
[CV] .

[Parallel(n_jobs=3)]: Done  28 tasks      | elapsed:    0.7s
[Parallel(n_jobs=3)]: Done  38 out of  70 | elapsed:    0.7s remaining:    0.6s
[Parallel(n_jobs=3)]: Done  50 out of  70 | elapsed:    0.8s remaining:    0.3s


[CV] alpha=100 .......................................................
[CV] .................. alpha=10, score=-0.419106291346, total=   0.0s
[CV] alpha=10 ........................................................
[CV] ................. alpha=100, score=-0.444480902582, total=   0.0s
[CV] alpha=100 .......................................................
[CV] .................. alpha=10, score=-0.395193245044, total=   0.0s
[CV] alpha=10 ........................................................
[CV] ................. alpha=100, score=-0.422893858518, total=   0.0s
[CV] alpha=100 .......................................................
[CV] .................. alpha=10, score=-0.377692129129, total=   0.0s
[CV] alpha=10 ........................................................
[CV] .................. alpha=100, score=-0.40697631266, total=   0.0s
[CV] .................. alpha=10, score=-0.401866179197, total=   0.0s
[CV] alpha=100 .......................................................
[CV] .

[Parallel(n_jobs=3)]: Done  70 out of  70 | elapsed:    0.9s finished


In [13]:
cv_score = np.array([-0.23824588, -0.20844122, -0.23356121, -0.21602981, -0.2285855,  -0.22002714,-0.21376681, -0.22638707, -0.21034281, -0.21976963])
cv_score.mean()

-0.221515708

In [14]:
nn_model.fit(data_X_encoded, data_y)


/Users/wqshen/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [15]:
#store the model
joblib.dump(nn_model, 'nn_model_without_d.pkl')
#to use saved model to predict
rf = joblib.load('nn_model_without_d.pkl')
rf.predict([x_test])

ValueError: shapes (1,8) and (28,1) not aligned: 8 (dim 1) != 28 (dim 0)